# Hola Mundo: Desde el fichero a la pantalla

## ¿Quién soy yo?

* Jesús Espino 

  * Twitter: @jespinog
  * Github: http://github.com/jespino
  * Email: jespinog@gmail.com
  
* Desarrollador en:

  <img src="http://www.bokzuy.com/images/logo_kaleidos_big.png" width="200" /> <img src="http://kaleidos.net/media/filer_public_thumbnails/filer_public/e5/14/e514887a-d582-44f6-9503-a892cd26a619/taiga-logo.jpg__300x300_q85_subsampling-2.jpg" width="200" />

## ¿De que vamos a hablar?
* Compiladores
* Interpretes
* Python
* Hola Mundo

## ¿Qué es un compilador?
* Convierte un código fuente en codigo ejecutable por una maquina
* Normalmente se divide en varias fases (analisis lexico, analisis sintactico, construcción de arbol abstracto, generación de código maquina)

## ¿Qué es un interprete?
* Lee un código fuente, y lo interpreta directamente.
* Va leyendo y ejecutando el código tal cual.

## ¿Que es python?
* ¿Es interpretado?
* ¿Es compilado?

<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />

## Ambas cosas

* Python compila el código a ByteCode
* El ByteCode es un código que interpreta la "Maquina Virtual" de python.

## Y ¿cómo lo hace?
![](compiler-graph.jpg)

##  Un paseo por el código de Python

* https://github.com/python/cpython/tree/3.6/

## Hola Mundo: Del fichero a la pantalla

In [ ]:
source_code = open("holamundo.py", "r").read()
print(source_code)

In [ ]:
import holamundo
holamundo.holamundo()

## El resultado del compilador

* El compilador genera ByteCode, veamos como

In [ ]:
c = compile(source_code, filename="holamundo.py", mode="exec")
print(type(c))
print(c.co_code)
print([b for b in bytes(c.co_code)])
print(c.co_code.hex())

## ¿Pero que demonios significa esto?

 * https://github.com/python/cpython/blob/3.6/Include/opcode.h
 * https://github.com/python/cpython/blob/3.6/Python/ceval.c#L1220

## ¿En serio? ¿Código fuente?

* Vaaaale... si lo prefieres tambien hay documentación:
  * https://docs.python.org/3.6/library/dis.html#python-bytecode-instructions

## Vamos a leerlo un poco mejor

In [ ]:
import dis
dis.dis(c)

In [ ]:
print(type(holamundo.holamundo))
print(type(holamundo.holamundo.__code__))
print(holamundo.holamundo.__code__.co_code)
print([b for b in bytes(holamundo.holamundo.__code__.co_code)])
print(holamundo.holamundo.__code__.co_code.hex())

In [ ]:
dis.dis(holamundo.holamundo)

## Y ¿Como lo ejecuto?

* Usando `exec` o `eval`
* `exec` ejecuta un bloque de código y no devuelve resultado
* `eval` ejecuta una expresion y devuelve resultado

In [ ]:
exec(c)
holamundo.holamundo()

## Pero ¿cómo llego hasta aquí?

* El compilado se divide principalmente en 5 fases:
    * Tokenizado.
    * Construcción del arbol sintaction (parseado).
    * Construcción del arbol sintaction abstracto (AST).
    * Construcción de la tabla de simbolos.
    * Traducción a ByteCode.

## El primer paso: Tokenizado

El primer paso es el tokenizado del codigo, que divide y etiqueta cada uno de los elementos de nuestro codigo para luego ser procesados.

In [ ]:
import tokenize

tokens = tokenize.tokenize(open('holamundo.py', "rb").readline)
print("\n".join([str(token) for token in tokens]))

### ¿Donde ocurre la magia?

* https://github.com/python/cpython/blob/3.6/Parser/tokenizer.c#L1363

* pgen genera el código necesario para la gramatica
* asdl_c.py genera el codigo necesario para la generación del AST

## Segundo paso: Analisis sintactico
Tras el tokenizado se realiza el analisis sintactico, que organiza nuestros tokens en un Syntax-Tree

In [ ]:
import parser
st = parser.suite(source_code)
st.totuple()

Podemos verlo un poco mejor si traducimos los IDS a Tokens y Simbolos

In [ ]:
import stpp
stpp.stpp(st)

In [ ]:
mod = st.compile()
exec(mod)
holamundo()

### ¿Donde ocurre la magia?

* pgen genera el código necesario para la gramatica:
  * https://github.com/python/cpython/blob/3.6/Grammar/Grammar
* https://github.com/python/cpython/blob/3.6/Parser/parsetok.c#L183

## Tercer paso: Analisis semantico
Tras la construcción del analisis sintactico se hace el analisis semantico y se construye un Abstract-Syntax-Tree

In [ ]:
import ast
x = ast.parse(source_code)
rint(ast.dump(x))

In [ ]:
import astpp
print(astpp.dump(ast.parse(source_code)))

In [ ]:
import symtable
import symtablepp
table = symtable.symtable(source_code, filename="holamundo.py", compile_type="exec")
symtablepp.symtablepp(table)

### ¿Donde ocurre la magia?

* asdl_c.py genera el codigo necesario para la generación del AST:
  * https://github.com/python/cpython/blob/3.6/Parser/Python.asdl
* https://github.com/python/cpython/blob/3.6/Python/ast.c#L761

## Resumen
![](compiler-graph.jpg)

## Dudas